In [76]:
import json
import re
import random

data = {}
transfer_stations = []

In [77]:
with open('subway.txt', 'r') as f:
    for line_num, l in enumerate(f):
        stations = re.split(r'\s+-\s+|\n', l.strip('\n'))

        for index, station in enumerate(stations):
            data.setdefault(station, {})
            data[station].setdefault(line_num + 1, [])

            if index > 0:
                data[station][line_num + 1].append((stations[index - 1], line_num + 1, random.randint(2, 5)))
            if index < len(stations) - 1:
                data[station][line_num + 1].append((stations[index + 1], line_num + 1, random.randint(2, 5)))

In [78]:
for station in data.keys():

    if len(data[station]) == 1:
        continue

    for line in data[station].keys():
        for transfer_line in data[station].keys():

            if line == transfer_line:
                continue

            data[station][line].append((station, transfer_line, random.randint(1, 4)))

In [79]:
with open('subway(unequal).json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)